How do we want to implement currencies at the moment?

1. \\$5 + 10 CHF = \\$10 if rate is 2:1.
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. Dollar/Franc duplication.
11. ~~Common equals~~.
12. **Common times**.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?

In [4]:
from abc import ABC, abstractmethod


def testMultiplication():
    # In Java, declare all as type Money first, then:
    five = Money.dollar(5)
    assert Money.dollar(10) == five.times(2)
    assert Money.dollar(15) == five.times(3)


def testEquality():
    assert Money.dollar(5) == Money.dollar(5)
    assert Money.dollar(5) != Money.dollar(6)
    assert Money.franc(5) == Money.franc(5)
    assert Money.franc(5) != Money.franc(6)
    assert Money.franc(5) != Money.dollar(5)


def testFrancMultiplication():
    # In Java, declare all as type Money first, then:
    five = Money.franc(5)
    assert Money.franc(10) == five.times(2)
    assert Money.franc(15) == five.times(3)


def testCurrency():
    assert "USD" == Money.dollar(1).currency()
    assert "CHF" == Money.franc(1).currency()
    

class Money(ABC):

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.__class__ is object.__class__


    @staticmethod
    def dollar(amount): # -> Money
        return Dollar(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Franc(amount, "CHF")
    

    @abstractmethod
    def times(self, multiplier):
        pass


    def currency(self):
        return self._currency
    

class Dollar(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)


    def times(self, multiplier) -> Money:
        return Money.dollar(self._amount * multiplier)
    

class Franc(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)


    def times(self, multiplier) -> Money:
        return Money.franc(self._amount * multiplier)


The two implementations of `times()` are close, but not identical.

In `Franc`, however, we know that the currency instance variable is always
`“CHF”`, so we can write:

In [10]:
class Money:

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.__class__ is object.__class__


    @staticmethod
    def dollar(amount): # -> Money
        return Dollar(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Franc(amount, "CHF")
    

    def times(self, multiplier):
        return None


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

class Dollar(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)


    def times(self, multiplier) -> Money:
        return Dollar(self._amount * multiplier, self._currency)
    

class Franc(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)


    def times(self, multiplier) -> Money:
        return Franc(self._amount * multiplier, self._currency)
    

testMultiplication()
testEquality()
testFrancMultiplication()
testCurrency()

The situation that we had was a `Franc(10, “CHF”)` and
a `Money(10, “CHF”)` that were reported to be not equal, even though we would like
them to be equal. We can use exactly this for our test:

In [11]:
def testDifferentClassEquality():
    assert Money(10, "CHF") == Franc(10, "CHF")
    assert Money(10, "USD") == Dollar(10, "USD")


testDifferentClassEquality()

AssertionError: 

It fails, as expected. The `equals()` code should compare currencies, not classes:

In [12]:
class Money:

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Dollar(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Franc(amount, "CHF")
    

    def times(self, multiplier):
        return None


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency
    

testDifferentClassEquality()

Now we can return a `Money` from `Franc.times()` and still pass the tests

In [13]:
class Dollar(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)


    def times(self, multiplier) -> Money:
        return Money(self._amount * multiplier, self._currency)
    

class Franc(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)


    def times(self, multiplier) -> Money:
        return Money(self._amount * multiplier, self._currency)
    

testDifferentClassEquality()

Yes! Now the two implementations are identical, so we can push them up.

In [15]:
class Money:

    _amount = None     # In Java, protected
    _currency = None   # In Java, protected


    def __init__(self, amount, currency):
        self._amount = amount
        self._currency = currency
    
    
    def __eq__(self, object):
        # In Java, casting to Money first, then:
        return self._amount == object._amount and self.currency() == object.currency()


    @staticmethod
    def dollar(amount): # -> Money
        return Dollar(amount, "USD")
    

    @staticmethod
    def franc(amount):  # -> Money
        return Franc(amount, "CHF")
    

    def times(self, multiplier):
        return Money(self._amount * multiplier, self._currency)


    def currency(self):
        return self._currency
    

    def __str__(self):
        return str(self._amount) + " " + self._currency


class Dollar(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)
    

class Franc(Money):


    def __init__(self, amount, currency):
        super().__init__(amount, currency)
    

testMultiplication()
testEquality()
testFrancMultiplication()
testCurrency()
testDifferentClassEquality()

1. \\$5 + 10 CHF = \\$10 if rate is 2:1.
2. ~~\\$5 * 2 = \\$10~~.
3. ~~make `amount` private~~.
4. ~~Dollar side-effects?~~
5. Money rounding?
6. ~~`equals()`~~.
7. Equal null.
8. Equal object.
9. ~~5 CHF * 2 = 10 CHF~~.
10. Dollar/Franc duplication.
11. ~~Common equals~~.
12. ~~Common times~~.
13. ~~Compare Francs with Dollars~~.
14. ~~Currency?~~
15. Delete testFrancMultiplication?

Multiplication in place, we are ready to eliminate the stupid subclasses.